In [1]:
import pandas as pd
import pyodbc
import sqlalchemy.types
from sqlalchemy import create_engine
from urllib import parse
from timeit import default_timer as timer

In [16]:
# Define Database Connection for PROD

CONNAS400 = """
Driver={iSeries Access ODBC Driver};
system=10.143.12.10;
Server=AS400;
Database=PROD;
UID=SMY;
PWD=SMY;
"""

In [26]:
# Define Database Connection for ccsdta

CONNAS400_2 = """
Driver={iSeries Access ODBC Driver};
system=10.143.12.10;
Server=AS400;
Database=CCSDTA;
UID=SMY;
PWD=SMY;
"""


In [23]:
sql_inv = """
    SELECT
    b.plt,
    b.majloc,
    b.minloc,
    y.itmid,
    b.qty,
    y.itmdesc,
    b.upddte,
    SUBSTR (Altdesc, 15, 1) Class,
    substr (altdesc, 12, 1) PRODCD,
    substr (altdesc, 1, 1) PFX,
    substr (altdesc, 7, 3) DEPT,
    substr (Altdesc, 16, 1) STOCK,
    substr (y.itmid, 1, 1) ITEM1,
    cast(((tcst02 + tcst03) * qty) / 1000 AS float) COST1,
    cast(((tcst05 + tcst06) * qty) / 1000 AS float) COST4,
    cast(((tcst08 + tcst09) * qty) / 1000 AS float) COST7,
    cast(((tcst11 + tcst12) * qty) / 1000 AS float) COST10
FROM
    DCSCIM y,
    DMFCMAR x,
    DCSILM b
WHERE
    x.itmid = y.itmid
    AND x.itmid = b.itmid
    AND x.plt = b.plt
    AND qty <> 0
    AND x.COSTID = 'FRZ'
    AND x.plt NOT IN ('53', '54', '55', '56', '59')

"""

In [25]:
print(sql_inv)


    SELECT
    b.plt,
    b.majloc,
    b.minloc,
    y.itmid,
    b.qty,
    y.itmdesc,
    b.upddte,
    SUBSTR (Altdesc, 15, 1) Class,
    substr (altdesc, 12, 1) PRODCD,
    substr (altdesc, 1, 1) PFX,
    substr (altdesc, 7, 3) DEPT,
    substr (Altdesc, 16, 1) STOCK,
    substr (y.itmid, 1, 1) ITEM1,
    cast(((tcst02 + tcst03) * qty) / 1000 AS float) COST1,
    cast(((tcst05 + tcst06) * qty) / 1000 AS float) COST4,
    cast(((tcst08 + tcst09) * qty) / 1000 AS float) COST7,
    cast(((tcst11 + tcst12) * qty) / 1000 AS float) COST10
FROM
    DCSCIM y,
    DMFCMAR x,
    DCSILM b
WHERE
    x.itmid = y.itmid
    AND x.itmid = b.itmid
    AND x.plt = b.plt
    AND qty <> 0
    AND x.COSTID = 'FRZ'
    AND x.plt NOT IN ('53', '54', '55', '56', '59')




In [29]:
dbcnxn = pyodbc.connect(CONNAS400_2)
cursor = dbcnxn.cursor()
start = timer()
try:
        cursor.execute(sql_inv)
        result = cursor.fetchall()
except Exception as e:
    msg = 'AS400 Employee Query Failed: ' + str(e)
    result = []
    print(msg)
    dbcnxn.close()
else:
    msg = str(len(result)) + " AS400 Employee Records Processed From Table"
print(msg)
print("AS400 Connect/Query Time = " + str(round((timer() - start), 3)) + " sec")

AS400 Employee Query Failed: ('42S02', '[42S02] [IBM][System i Access ODBC Driver][DB2 for i5/OS]SQL0204 - DCSCIM in SMY type *FILE not found. (-204) (SQLExecDirectW)')
AS400 Employee Query Failed: ('42S02', '[42S02] [IBM][System i Access ODBC Driver][DB2 for i5/OS]SQL0204 - DCSCIM in SMY type *FILE not found. (-204) (SQLExecDirectW)')
AS400 Connect/Query Time = 0.057 sec


#### Build Dataframe for Component Inventory

In [ ]:
df_inv = pd.DataFrame.from_records(result_inv)
data_type_dict = {'PLT': str, 'ITMID': str, 'QTY': int, 'CLASS': str, 'ITMDESC': str}
df_inv.columns = ['PLT', 'ITMID', 'QTY', 'CLASS', 'ITMDESC']
df_inv = df_inv.dropna()
df_inv = df_inv.astype(data_type_dict)
df_inv = df_inv.convert_dtypes()


In [ ]:
def get_inv():
    dbcnxn = pyodbc.connect(CONNAS400_2)
    cursor_as400 = dbcnxn.cursor()

In [23]:
def get_emps():
    """Get Employees From iSeries AS400"""
    dbcnxn = pyodbc.connect(CONNAS400)
    cursor = dbcnxn.cursor()
    eng_login = ['9999', 'ELMER J FUDD', 'ENG']
    lead_login = ['1208', 'Wile E Coyote', 'ENG']
    eng = ['1208', '9107', '1656', '1472', '1626', '1351']

    strsql = """SELECT STRIP(EMP_CLOCK_NUMBER) As Clock,
                    CONCAT(CONCAT(STRIP(EMP_FIRST_NAME), ' '),
                    STRIP(EMP_LAST_NAME)) As Name,
                    STRIP(EMP_POSITION_CODE) As Code
            FROM PROD.FPCLCKPAY
            WHERE (EMP_LOCATION = 09) AND (EMP_LAST_NAME <> 'TEMP') AND (EMP_SHIFT_TYPE = 'A')
            ORDER BY EMP_CLOCK_NUMBER"""
    try:
        cursor.execute(strsql)
        result = cursor.fetchall()
    except Exception as e:
        msg = 'AS400 Employee Query Failed: ' + str(e)
        result = []
        print(msg)
    else:
        msg = str(len(result)) + " AS400 Employee Records Processed From Table"
        print(msg)
    result.append(eng_login)
    result.append(lead_login)
    for row in result:
        if row[0] in eng:
            row[2] = 'ENG'
    dbcnxn.close()
    return result

In [3]:
print(get_emps())


319 AS400 Employee Records Processed From Table
[('1001', 'DEBBIE POMEROY-GAMMONS', 'DEF'), ('1013', 'BETH THOMPSON', 'DEF'), ('1032', 'THOMAS BOGLE', 'DEF'), ('1039', 'VICKIE ANDERSON', 'DEF'), ('1090', 'BILLY NIPPERS', 'DEF'), ('1122', 'SHELLEY HOUSTON', 'DEF'), ('1139', 'ANTIONETTE BASS', 'DEF'), ('1175', 'WYOMIA CLARK', 'DEF'), ('1176', 'NALEE NHOTSAVANG', 'DEF'), ('1177', 'CHARLES TOOMBS', 'DEF'), ('1213', 'DOUGLAS SHIPP', 'DEF'), ('1238', 'MARICHU RIVERA', 'DEF'), ('1258', 'PHOUVEUNE LAMNGEUN', 'DEF'), ('1318', 'TERESA HOLLIS', 'DEF'), ('1331', 'NANCY CLARK', 'DEF'), ('1332', 'BRENDA CUNNINGHAM', 'DEF'), ('1347', 'HERBERT GRAHAM', 'DEF'), ('1352', 'DALE GERHART', 'DEF'), ('1372', 'SITHONH BOUAPHACHANH', 'DEF'), ('1429', 'GARY BELCHER', 'DEF'), ('1444', 'RILEY BREWER  JR.', 'DEF'), ('1464', 'CHRISTOPHER HIX', 'DEF'), ('1483', 'BARRY KIRBY', 'DEF'), ('1496', 'TERRY MARTIN', 'DEF'), ('1499', 'ACKHASONE PHILAVARN', 'DEF'), ('1514', 'KHAMIN VONGPANYA', 'DEF'), ('1515', 'PHILLIP STINSO